<a href="https://colab.research.google.com/github/profjefer/ANPRwithPython/blob/main/train_car_horn_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Car Horn Detection

## Get the Dataset
- TODO:
  * <input type="checkbox" checked> Download the dataset.
  * <input type="checkbox" checked> Extract all the parts
  * <input type="checkbox"> Read the files and preprocess them
  * <input type="checkbox"> Dump in a Numpy pickle (.npy) file.

### Download the dataset

In [ ]:
!wget https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz -O urban8k.tgz

--2018-10-19 04:54:26--  https://zenodo.org/record/1203745/files/UrbanSound8K.tar.gz
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6023741708 (5.6G) [application/octet-stream]
Saving to: ‘urban8k.tgz’

urban8k.tgz         100%[===================>]   5.61G  10.8MB/s    in 6m 43s  

2018-10-19 05:01:16 (14.2 MB/s) - ‘urban8k.tgz’ saved [6023741708/6023741708]



### Extract it

In [ ]:
!tar -xzf urban8k.tgz

### Have a look at dataset files

In [ ]:
!cat UrbanSound8K/UrbanSound8K_README.txt

UrbanSound8K

Created By
----------

Justin Salamon*^, Christopher Jacoby* and Juan Pablo Bello*
* Music and Audio Research Lab (MARL), New York University, USA
^ Center for Urban Science and Progress (CUSP), New York University, USA
http://serv.cusp.nyu.edu/projects/urbansounddataset
http://marl.smusic.nyu.edu/
http://cusp.nyu.edu/

Version 1.0


Description
-----------

This dataset contains 8732 labeled sound excerpts (<=4s) of urban sounds from 10 classes: air_conditioner, car_horn, 
children_playing, dog_bark, drilling, engine_idling, gun_shot, jackhammer, siren, and street_music. The classes are 
drawn from the urban sound taxonomy described in the following article, which also includes a detailed description of 
the dataset and how it was compiled:

J. Salamon, C. Jacoby and J. P. Bello, "A Dataset and Taxonomy for Urban Sound Research", 
22nd ACM International Conference on Multimedia, Orlando USA, Nov. 2014.

All excerpts are taken from field recordings uploaded to www.freesou

In [ ]:
!dir UrbanSound8K/audio/fold1

101415-3-0-2.wav   146186-5-0-8.wav    177621-0-0-145.wav  40722-8-0-5.wav
101415-3-0-3.wav   146186-5-0-9.wav    177621-0-0-147.wav  40722-8-0-6.wav
101415-3-0-8.wav   147764-4-0-0.wav    177621-0-0-27.wav   40722-8-0-7.wav
102106-3-0-0.wav   147764-4-1-0.wav    177621-0-0-2.wav    40722-8-0-8.wav
102305-6-0-0.wav   147764-4-2-0.wav    177621-0-0-31.wav   43786-3-0-0.wav
102842-3-0-1.wav   147764-4-3-0.wav    177621-0-0-35.wav   43786-3-1-0.wav
102842-3-1-0.wav   147764-4-4-0.wav    177621-0-0-46.wav   43806-8-0-0.wav
102842-3-1-5.wav   147764-4-4-1.wav    177621-0-0-47.wav   44736-5-0-0.wav
102842-3-1-6.wav   147764-4-4-2.wav    177621-0-0-48.wav   44736-5-0-1.wav
103074-7-0-0.wav   147764-4-5-0.wav    177621-0-0-54.wav   46654-6-0-0.wav
103074-7-0-1.wav   147764-4-6-0.wav    177621-0-0-57.wav   46656-6-0-0.wav
103074-7-0-2.wav   147764-4-7-0.wav    177621-0-0-80.wav   46656-6-1-0.wav
103074-7-1-0.wav   147764-4-8-0.wav    177621-0-0-88.wav   46656-6-2-0.wav
103074-7-1-1.wav   150341

In [ ]:
import pandas as pd
metadata = pd.read_csv('UrbanSound8K/metadata/UrbanSound8K.csv')
metadata.drop(columns=['fsID', 'fold', 'classID'], inplace=True)

### Explore the metadata

In [ ]:
metadata.groupby('class').slice_file_name.count()

class
air_conditioner     1000
car_horn             429
children_playing    1000
dog_bark            1000
drilling            1000
engine_idling       1000
gun_shot             374
jackhammer          1000
siren                929
street_music        1000
Name: slice_file_name, dtype: int64

### Load an audio file

In [ ]:
import tensorflow as tf
from tensorflow.contrib.framework.python.ops import audio_ops

In [ ]:
audio_binary = tf.read_file('UrbanSound8K/audio/fold1/118279-8-0-0.wav')
desired_channels=1
wav_decoder = audio_ops.decode_wav(audio_binary, desired_channels=desired_channels)

audio = wav_decoder.audio
audio = tf.reshape(wav_decoder.audio, [1, tf.shape(audio)[0]])

audio_stft = tf.contrib.signal.stft(audio, frame_length=1024, frame_step=512, fft_length=1024)

In [ ]:
with tf.Session() as sess:
  print(sess.run(audio_stft))

[[[ 2.51935124e-01+0.00000000e+00j -3.34782720e-01+2.92894316e+00j
   -3.63964289e-01-1.68518007e+00j ... -4.00841236e-05-9.02414322e-05j
   -2.33650208e-05+1.09791756e-04j  1.11445785e-04+0.00000000e+00j]
  [-4.96557808e+00+0.00000000e+00j  4.66363907e+00-1.99526048e+00j
   -2.98943686e+00-2.19562531e-01j ...  1.34706497e-04-1.16579235e-04j
   -3.33786011e-05+8.80360603e-05j  1.74045563e-05-0.00000000e+00j]
  [-2.48523331e+00+0.00000000e+00j  1.81372666e+00+1.01712573e+00j
    2.68789351e-01+8.81789505e-01j ... -4.74601984e-05-5.39720058e-04j
    1.09016895e-04+2.60710716e-04j -1.81555748e-04-0.00000000e+00j]
  ...
  [ 2.23110271e+00+0.00000000e+00j -3.05308914e+00+2.02171803e+00j
    1.98752439e+00-1.28285336e+00j ... -1.51991844e-04+5.54323196e-06j
    3.22818756e-04-1.53779984e-05j -3.58819962e-04+0.00000000e+00j]
  [-6.42441034e+00+0.00000000e+00j  4.77360535e+00-3.29090977e+00j
   -1.06434330e-01+2.44245386e+00j ...  5.91166317e-05-9.23871994e-05j
   -8.51154327e-05+1.19805336e-0